## A transformada discreta de Fourier

A FFT (Fast Fourier Transform, transformada rápida de Fourier) é  considerado um dos algoritmos mais importantes de sempre. Tem ordem ${\mathcal O}(N\log N)$, quando de um modo simplista parece uma computação ${\mathcal O}(N^2)$

A DFT (Discrete Fourier Transform), tal como a transformada de Fourier, tem também a sua inversa, definidas como:

Forward Discrete Fourier Transform (DFT):
$$ 
X_k = \sum_{n=0}^{N-1} x_n e^{-i 2 \pi k n /N}
$$

Inverse Discrete Fourier Transform (IDFT):
$$ 
x_n = \frac{1}{N} \sum_{k=0}^{N-1} X_k e^{ i 2 \pi k n /N}
$$

Esta transformação, do chamado espaço das configurações ou real, para o espaço das frequências, é muito útil em vários cenários, permitindo analisar sinais, ou resolver problemas doutro modo mais difíceis.

Dada a sua importância há várias implementações, sendo a mais rápida (?) a FFTW, que tem versão em Python via o pacote PyFFTW. Usaremos em vez disso a versão existente no Numpy ou Scipy, que são "wrappers" para a biblioteca FFTPACK (escrita em Fortran), e encontradas nos submódulos numpy.fft e scipy.fftpack, respectivamente.

## Cálculo da Transformada de Fourier Discreta (DFT)

A definição acima pode ser escrita como uma operação matricial (já que não é mais que um conjunto de operações lineares)
$$
\overrightarrow{X} =M \cdot \overrightarrow{x},
$$
em que a matriz $ M$ é dada por 
$$ 
M_{kn} = e^{-i 2 \pi k n /N}.
$$
Assim a DFT pode ser calculada, por exemplo, com o código abaixo:

In [1]:
import numpy as np
def DFT_lenta(x):
    """Calcula a Transformada de Fourier discreta de um array 1D, x"""
    x = np.asarray(x, dtype=float)
    N = x.shape[0]
    n = np.arange(N)
    k = n.reshape((N, 1))
    M = np.exp(-2j * np.pi * k * n / N)
    return np.dot(M, x)

O resultado pode ser comparado com o obtido pela rotina do numpy, para testar:

In [2]:
x = np.random.random(1024)
np.allclose(DFT_lenta(x), np.fft.fft(x))

True

Outro ponto a ter em atenção é o tempo que demora. Podemos comparar os tempos de execução das duas via:

In [3]:
#import timeit 
%timeit DFT_lenta(x)
%timeit np.fft.fft(x)

17.6 ms ± 419 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
6.3 µs ± 13.5 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


Bem mais lento! 3 ordens de grandeza, na verdade! Se em vez de 1024 elementos tivessemos $N=10^6$ a fft demoraria uns 40 ms ($15 \mu s \times 10^3\log(10^3$)), mas a nossa rotina demoraria perto de 18 horas ($70 ms \times (10^3)^2$)! 

De onde vem toda esta discrepância? O que acontece é que o algoritmo de Cooley-Tukey explora simetrias para reduzir (muito!) o número de cálculos a efectuar.

### Simetrias na DFT
Calculemos $X_{N+k}$.
$$
\begin{align}
X_{N+k} &= \sum_{n=0}^{N-1} x_n e^{-i 2 \pi (N+k) n /N}\\
        &= \sum_{n=0}^{N-1} x_n e^{-i 2 \pi k n /N} \cdot e^{-i 2 \pi N n /N}\\
        &= \sum_{n=0}^{N-1} x_n e^{-i 2 \pi k n /N}
\end{align}
$$
A última linha mostra uma importante propriedade da DFT:

$$
X_{N+k}=X_k.
$$

Uma simples extensão leva a,

$$
X_{k+i \cdot N}=X_k
$$

para qualquer inteiro i. Como veremos de seguida, esta simetria pode ser usada para calcular a DFT muito mais depressa.



### De DFT para FFT: explorando a simetria

Cooley e Tukey mostraram que é possível dividir o cálculo da DFT em duas partes menores. Da definição anterior de DFT temos:

$$
\begin{align}
X_k  &=\sum_{n=0}^{N-1} x_n e^{-i 2 \pi k n /N}  \\
     &=\sum_{m=0}^{N/2-1}x_{2m}\cdot e^{−i 2 \pi k (2m) / N} +\sum_{m=0}^{N/2-1}x_{2m+1} \cdot e^{−i 2 \pi k (2m+1) /N}\\
     & =\sum_{m=0}^{N/2-1}x_{2m}\cdot e^{−i 2 \pi k m / (N/2)} + e^{−i 2 \pi k / N}\sum_{m=0}^{N/2-1}x_{2m+1} \cdot e^{−i 2\pi k m / (N/2)}
\end{align}
$$

Assim dividimos uma DFT em dois termos, cada um dos quais se parece muito com uma DFT, mais pequena, uma sobre os valores ímpares e outra sobre os valores pares. Para já ainda não poupamos nada, pois cada termo leva a $(N/2)N$ cálculos, para um total de $N^2$. O truque é usar simetrias em cada um destes termos. Porque k varia em $0\leq k<N$, enquanto a gama de n é $0\leq n<M\equiv N/2$, as propriedades de simetria da secção anterior mostram-nos que só precisamos de fazer metade dos cálculos para cada sub-problema. O problema ${\mathcal O}(N^2)$ tornou-se ${\mathcal O}(M^2)$, com M metade de N.

Mas podemos continuar a dividir o problema, enquanto M for par, passando o custo computacional a metade de cada vez. No limite assimptótico, este processo recursivo escala como ${\mathcal O}(N\log N)$.

O algoritmo recursivo pode ser facilmente implementado em Python. Quando o tamanho dos subproblemas for pequeno, podemos retornar a usar nosso código anterior (DFT_lenta):


In [4]:
def FFT(x):
    """Implementação recursiva do algoritmo FFT em 1D de Cooley-Tukey"""
    x = np.asarray(x, dtype=float)
    N = x.shape[0]
    
    if N % 2 > 0:
        raise ValueError("tamanho de x deve ser potência de 2")
    elif N <= 32:  # para não perder tempo quando é muito curta. Não é estritamente necessária.
        return DFT_lenta(x)
    else:
        X_par = FFT(x[::2])
        X_impar = FFT(x[1::2])
        factor = np.exp(-2j * np.pi * np.arange(N) / N)
        
        return np.concatenate([X_par + factor[:N // 2] * X_impar,
                               X_par + factor[N // 2:] * X_impar])

De novo é conveniente verificar que os resultados são os esperados:

In [5]:
y = np.random.random(1024)
np.allclose(FFT(y), np.fft.fft(y))

True

E também verificar se estamos no caminho certo em termos de produzir um código mais rápido:

In [6]:
%timeit DFT_lenta(x)
%timeit FFT(x)
%timeit np.fft.fft(x)

17.4 ms ± 150 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
935 µs ± 1.87 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
6.43 µs ± 9.61 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


Ganhamos mais de uma ordem de grandeza, para o problema pequeno! Melhor ainda, esta versão escala como ${\mathcal O}(N\log N)$: temos a nossa própria implementação da Fast Fourier Transform.

É claro que também percebemos que ainda estamos longe da FFT do numpy. Isso não é de estranhar. Essa rotina do FFTPACK, escrita em Fortran, tem anos e anos de afinações e optimizações. Ainda por cima a nossa implementação envolve o stack do Python e alocação de arrays temporários, o que adiciona tempo de computação de um modo significativo.

Uma estratégia adicional, útil sempre que trabalhamos com Numpy, é vectorizar as operações sobre arrays (isto é, o computador "percebe" que vai fazer a mesma coisa muitas vezes sobre elementos independentes, e tenta fazê-las em paralelo). Fazendo isto eliminamos também as chamadas funções recursivas.


### Versão Numpy Vectorizada

Note-se que na implementação recursiva anterior, no nível mais baixo fazemos $N/32$ produtos matriz-vector idênticas ( o 32 aqui é arbitrário!). A eficiência do algoritmo teria tudo a ganhar ao fazer estas operações todas em "simultâneo", como um único produto matriz-matriz. Em cada subnível subsequente da recursão, também fazemos operações duplicadas, que podem ser vectorizadas. O Numpy é muito bom neste tipo de operações, e podemos usar essa funcionalidade para escrever o código seguinte:



In [7]:
def FFT_vectorizada(x):
    """Versão vectorizada, não recursiva, da FFT de Cooley-Tukey"""
    x = np.asarray(x, dtype=float)
    N = x.shape[0]

    if N % 2 > 0:
        raise ValueError("tamanho de x deve ser potência de 2")

    # N_min aqui é equivalente à condição de paragem acima,
    # e deve ser uma potência de 2. Não é estritamente necessária.
    N_min = min(N, 32)
    
    # Faz uma DFT de O[N^2] em todos os sub-problemas de tamanho N_min de uma vez.
    n = np.arange(N_min)
    k = n[:, None]
    M = np.exp(-2j * np.pi * n * k / N_min)
    X = np.dot(M, x.reshape((N_min, -1)))

    # constrói cada nível do cálculo recursivo de uma vez
    while X.shape[0] < N:
        X_par = X[:, :X.shape[1] // 2]
        X_impar = X[:, X.shape[1] // 2:]
        factor = np.exp(-1j * np.pi * np.arange(X.shape[0])
                        / X.shape[0])[:, None]
        X = np.vstack([X_par + factor * X_impar,
                       X_par - factor * X_impar])

    return X.ravel()

Embora o algoritmo fique mais difícil de seguir (eu sei que disse que é de evitar, mas nas ferramentas mais básicas, frequentemente usadas, e já bem testadas, vale a pena!) só fizemos rearranjos das operações da versão anterior, com uma excepção: exploramos a simetria no cálculo da quantidade `factor` e construímos apenas metade do array.

Verifiquemos então mais uma vez que os resultados estão correctos:

In [8]:
x = np.random.random(1024)
np.allclose(FFT_vectorizada(x), np.fft.fft(x))

True

Como os algoritmos estão a ficar mais eficientes, vamos aumentar o tamanho do array x (retirando a versão DFT_lenta, para não atrasar):

In [9]:
x = np.random.random(1024 * 16)
%timeit FFT(x)
%timeit FFT_vectorizada(x)
%timeit np.fft.fft(x)

16 ms ± 35.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
1.14 ms ± 47.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
131 µs ± 3.12 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


Melhoramos o desempenho por mais uma ordem de grandeza! Estamos apenas a um factor de 10 da implementação da FFTPACK, e isto com apenas uma dúzia de linhas de Python puro+Numpy. É evidente qual devemos usar em termos computacionais, mas em termos de legibilidade, pode comparar: o código fonte da versão do FFTPACK pode ser visto [aqui](http://www.netlib.org/fftpack/fft.c) (versão em C).